# Projeto de Análise de Desigualdade em Saúde no modelo HEAT/OMS
## Bloco 2025/1 = análise do sistema de mortalidade do SUS com agregação por unidades federativas




#### Aspectos gerais e participantes

Projeto Desenvolvido em parceria da MS/SECTICS/DESID com a Organização Mundial de Saúde

Elaborado em linguagem Python, utilizando a biblioteca pysus para acesso ao SIM (Sistema de Informações de Mortalidade do SUS)

Coordenador-geral:

Coordenadores: 

Cientista de dados: Marcos Mesquita

Outros participantes:

Data: 06/12/2024


#### Objetivos

O principal objetivo desse projeto é realizar a avaliação de desigualdade na saúde no Brasil com ênfase para a dimensão de unidades subnacionais, estados e municípios brasileiros, a partir da aplicação do modelo do HEAT, tendo como objetivos específicos:

- Apresentar indicadores de saúde no Brasil desagregados por subunidades nacionais em uma visualização interativa a partir do modelo do HEAT Plus;
- Oferecer um repositório comum e atualizado com dados de desigualdade da saúde no Brasil desagregados por estados e municípios em um modelo sob reconhecimento internacional;
- Permitir exploração e acesso dos dados e da metodologia para outros tipos de análises.

# Etapa 1 - Definição do Bloco

O projeto ocorrerá em blocos sucessivos, nos quais serão definidos o problema e os indicadores a serem analisados.

### Definição do problema do bloco



 H0 - as unidades federativa são variáveis determinantes para mortalidade no país

### Escopo 


* localização: todo o Brasil, com agregação de mortalidade por Estados e Municípios
* período: 2018 a 2022 (a base SIM tem dados até 2022)?
* doenças: ?? (só “doenças evitáveis”?)
* outros critérios de filtro: até 74 anos de idade (caso seja aplicado o critério de "doenças evitáveis", aplica-se em consequência o limite de idade de até 74 anos; caso se aplica a metodologia brasileira, ainda se aplica o limite mínimo de idade de acima de 5 anos)

### Principais Problemas/Decisões

Ao longo do projeto devem ser feitas algumas escolhas que impactam todo o projeto e algumas destas merecem destaque até pela exigência de trabalho envolvida para tomada de decisão.

* CID 
    op1 - doeças evitáveis da MS/SVS 
    op2 - 
* critério de idade

* população
    op1 - estimativa por ano do IBGE (por municípios, mas sem outras veriáveis de desigualdade, como sexo e idade)
    op2 - projeção por ano do IBGE (por UF, tem variáveis de sexo e faixa etária, mas não tem dados por município)
    op3 - 

# Etapa 2 - Análise primária dos dados por bloco

## Coleta e estruturação dos dados

### Aspectos teóricos e téccnicos das fontes


#### Aspectos técnicos - ambiente, linguagem e ferramentas principais

A análise será feita utilizando a linguagem Python e a biblioteca Pysus que permite acesso às principais bases do SUS de forma direta.
O Pysus exige um ambiente linux para execução de alguns comandos o que foi possível, mesmo em uma máquina Windows, a partir do WSL do Windows.
Foi criado um ambiente em WSL e o conjunto da extração, tratamento e análise dos dados foi realizado no formato Notebook em um editor VSCode.


#### Morte e mortalidade no HEAT

Para entendermos como o HEAT trata a mortalidade, fizemos uma busca de indicadores de mortalidade no WHO Health Inequality Data Repository.

O repositório traz dados e metadados de todos os indicadores utilizados pelo HEAT.
Apresenta um total de 3915 indicadores, distribuídos entre 14 seções.

Foi feita uma revisão geral dos indicadores relacionados à mortalidade, que depois foi refinada pela busca das seguintes palavras : “death”, “mortality”, “fatal”, ‘fatality” ,“Probability of dying”, “suicide”. E foram encontrados 388 indicadores que trazem uma destas expressões na sua descrição.


#### SIM (Sistema de Informações de Mortalidade do SUS)

A mortalidade e os indicadores de morte serão buscados no SIM (Sistema de Informações de Mortalidade do SUS).
O SIM disponibiliza dados de mortes no Brasil desde 1996 por ocorrências, gerido pelo Departamento de Análise de Situação de Saúde, da Secretaria de Vigilância em Saúde do Ministério da Saúde, em conjunto com as Secretarias Estaduais e Municipais de Saúde. Estas últimas responsáveis efetivamente pela coleta dos dados.

O SIM traz informações do indivíduo, como sexo, idade, localidade e causas da morte, classificadas de acordo com o CID (Classificação Internacional de Doenças e Problemas Relacionadas à Saúde).


#### CID (Classificação Internacional de Doenças e Problemas Relacionadas à Saúde)

Com o intuito de padronizar a nomenclatura de patologias e com abrangência para todo o mundo, foi elaborado pela Organização Mundial da Saúde (OMS) uma  classificação internacional de doenças, a CID (Classificação Internacional de Doenças e Problemas Relacionadas à Saúde). 

A Classificação é atualizada periodicamente. A versão mais atual é a CID11, publicada em janeiro de 2022.

No entanto, por se tratar de revisão que traz impactos nos registros e que exige considerável atualização das ferramentas de captura das informações, o SIM utiliza versões anteriores da CID.
- CID9 - dados até 1995
- CID10 - a partir de 1996 até a data atual

Em razão do recorte temporal dos dados, de 1998 a 2022, iremos utilizar somente a CID10.
A CID10 é organizada em 22 capítulos e possui, além da classificação de doenças, descrição de códigos que permitem identificar sinais, sintomas, queixas, causas externas e circunstâncias sociais.


#### CID de doenças evitáveis

Como elemento do escopo deste trabalho iremos tratar de doenças evitáveis.

O Brasil, a partir da Secretaria de Vigilância em Saúde apresenta uma classificação de doenças evitáveis. 

Lista de causas evitáveis pode ser encontrada nos links: http://tabnet.datasus.gov.br/cgi/sim/Obitos_Evitaveis_0_a_4_anos.pdf




#### População Estimada


Os dados de população estimada serão obtidos a partir dos dados disponibilizdaos pelo Banco de Tabelas Estatísticas do IBGE, SIDRA. 

O SIDRA permite um acesso das bases via API, sidrapy.

Encontramos a informação da API de População Estimada no endereço: https://sidra.ibge.gov.br/tabela/6579

Os dados de população estimada tem granularidade município.

Iremos utilizar a informação de População Estimada por estar apresentar valores por municípios, contendo valores de 1998 a 2021. A SIDRA não possui valores de estimativas da população para o ano de 2022 em diante. 

Para o ano de 2022 iremos utilizar então o mesmo valor de 2021.




#### População Projeção IBGE


Os dados de projeção de população tem informações de sexo e idade, mas não tem granularidade por município.

A identificação de variáveis do API pode ser pesquisado no site.

https://servicodados.ibge.gov.br/api/docs/agregados?versao=3#api-acervo





### Coleta dos dados brutos

#### Instalação de bibliotecas

In [1]:
## Costuma ser exigida a instalação destas bibliotecas no primeiro acesso ao Pysus
# !apt-get update  
# !apt-get install libffi-dev  # Install libffi-dev
# !pip install --upgrade pip  # Ensure pip is up to date
# !pip install pysus # Exa

#### Importação de bibliotecas

In [1]:
# importar biblioteca básica para manipulação de dados
import pandas as pd


In [2]:
# importar bibliotecas para carga dos dados
import pysus
import os
import sidrapy
import requests # para carga de endereços web - api
from pysus import SIM
from pysus.online_data.SIM import download
from pysus.preprocessing.decoders import translate_variables_SIM
from pysus.preprocessing.SIM import group_and_count, redistribute_missing, redistribute_cid_chapter
from pysus.online_data.SIM import get_CID9_table, get_CID10_table, get_municipios, get_ocupations
from ftplib import FTP
from pathlib import Path  

#### Diretórios - configuração

In [4]:
# Definir diretório padrão para pysus
pysus.online_data.SIM.CACHEPATH="./downloads_sim_original"
# Definir pasta para download de arquivos csv e parquet
import os 

# Define diretório para armazenar arquivos do sim (carregados por pysus) quando o SIM.CACHEPATH não funcionar
os.makedirs('downloads_sim_original', exist_ok=True)  
downloads_sim = "downloads_sim_original"

# Define diretório para armazenar outros arquivos
os.makedirs('downloads_outros', exist_ok=True)  

#### Acesso ao SIM

In [5]:
sim = SIM().load() # Loads the files from DATASUS

#### Carga SIM principal

In [14]:
# Caso seja necessário trazer dados do SIM por ano (acrescentar algum ano)
# grupo = "CID10" # Agrupamento por CID10
# uf = "BR" # A sigla BR carrega dados de todos as unidades federativas
# ano = 2017 # Anos do escopo, de 2018 a 2022
# local = downloads_sim #
# dobr = download(grupo,uf,ano,local)

0it [00:00, ?it/s]


In [5]:
# Carrega os dados do SIM
# Definição de variáveis
grupo = "CID10" # Agrupamento por CID10
uf = "BR" # A sigla BR carrega dados de todos as unidades federativas
ano = [2018,2019,2020,2021,2022] # Anos do escopo, de 2018 a 2022
local = downloads_sim # Local para armazenar os arquivos, definido nas etapas anteriores

dobr = download(grupo,uf,ano,local)

DOBR2022.parquet: 100%|██████████| 3.78M/3.78M [01:21<00:00, 46.6kB/s]


#### Carga SIM auxiliar

O SIM traz algumas tabelas auxiliares de CID10 e municípios. Trazemos estas informações para algum tipo de batimento, mas para estas informações iremos utilizar fontes externas.

In [5]:
# Download de tabela auxiliar de CID10
cid10 = get_CID10_table()
cid10.head()

2024-12-20 19:50:23.507 | DEBUG    | pysus.online_data.SIM:get_CID10_table:116 - Stablishing connection with ftp.datasus.gov.br.
220 Microsoft FTP Service
2024-12-20 19:50:23.543 | DEBUG    | pysus.online_data.SIM:get_CID10_table:120 - Changing FTP work dir to: /dissemin/publicos/SIM/CID10/TABELAS
2024-12-20 19:50:24.156 | INFO     | pysus.online_data.SIM:get_CID10_table:146 - Data stored as parquet at /home/codespace/pysus/SIM_CID10_.parquet
2024-12-20 19:50:24.157 | DEBUG    | pysus.online_data.SIM:get_CID10_table:149 - CID10.DBF removed


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14257 entries, 0 to 14256
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   CID10      14257 non-null  object
 1   OPC        14257 non-null  object
 2   CAT        14257 non-null  object
 3   SUBCAT     14257 non-null  object
 4   DESCR      14257 non-null  object
 5   RESTRSEXO  14257 non-null  object
dtypes: object(6)
memory usage: 668.4+ KB
None


In [8]:
# Download de tabela auxiliar de CID10
munic = get_municipios()
print(munic.head())

2024-12-20 19:51:35.353 | DEBUG    | pysus.online_data.SIM:get_municipios:208 - Stablishing connection with ftp.datasus.gov.br.
220 Microsoft FTP Service
2024-12-20 19:51:35.380 | DEBUG    | pysus.online_data.SIM:get_municipios:212 - Changing FTP work dir to: /dissemin/publicos/SIM/CID10/TABELAS
2024-12-20 19:51:36.229 | INFO     | pysus.online_data.SIM:get_municipios:238 - Data stored as parquet at /home/codespace/pysus/SIM_CADMUN_.parquet
2024-12-20 19:51:36.229 | DEBUG    | pysus.online_data.SIM:get_municipios:241 - CADMUN.DBF removed


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5652 entries, 0 to 5651
Data columns (total 28 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   MUNCOD      5652 non-null   object 
 1   MUNCODDV    5652 non-null   object 
 2   SITUACAO    5652 non-null   object 
 3   MUNSINP     5652 non-null   object 
 4   MUNSIAFI    5652 non-null   object 
 5   MUNNOME     5652 non-null   object 
 6   MUNNOMEX    5652 non-null   object 
 7   OBSERV      5652 non-null   object 
 8   MUNSINON    5652 non-null   object 
 9   MUNSINONDV  5652 non-null   object 
 10  AMAZONIA    5652 non-null   object 
 11  FRONTEIRA   5652 non-null   object 
 12  CAPITAL     5652 non-null   object 
 13  UFCOD       5652 non-null   object 
 14  MESOCOD     5652 non-null   object 
 15  MICROCOD    5652 non-null   object 
 16  MSAUDCOD    5652 non-null   object 
 17  RSAUDCOD    5652 non-null   object 
 18  CSAUDCOD    5652 non-null   object 
 19  RMETRCOD    5652 non-null  

#### Carga externa ao SIM

Iremos carregar informações de fontes externas.



##### CIDs doenças evitáveis - OCDE

In [30]:
# Exibição de informações da tabela de CID evitáveis de acordo com a OCDE
cid_evit_ocde = pd.read_excel('downloads_outros/OCDE_avoidable_mortality.xlsx')
cid_evit_ocde = cid_evit_ocde.rename(columns={'Avoidable mortality':'CID_AVOID', 'ICD.IO':'CID_PAI'})
cid_evit_ocde.head()

,CID_AVOID,Preventable mortality,Treatable mortality,Group,Causes of deaths,Rationale for inclusion,Coluna2,CID_PAI
0,Avoidable mortality,Preventable mortality,NaN,Infectious diseases,Intestinal diseases,Most of these infections can be prevented thro...,(A00-A09),A0
1,Avoidable mortality,Preventable mortality,NaN,Infectious diseases,"Diphtheria, Tetanus, Poliomyelitis",Most of these infections can be prevented thro...,"(A35, A36, A80)",A35
2,Avoidable mortality,Preventable mortality,NaN,Infectious diseases,"Diphtheria, Tetanus, Poliomyelitis",Most of these infections can be prevented thro...,"(A35, A36, A80)",A36
3,Avoidable mortality,Preventable mortality,NaN,Infectious diseases,"Diphtheria, Tetanus, Poliomyelitis",Most of these infections can be prevented thro...,"(A35, A36, A80)",A80
4,Avoidable mortality,Preventable mortality,NaN,Infectious diseases,Whooping cough,Most of these infections can be prevented thro...,(A37),A37


In [32]:
cid_evit_ocde.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346 entries, 0 to 345
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   CID_AVOID                346 non-null    object
 1   Preventable mortality    189 non-null    object
 2   Treatable mortality      184 non-null    object
 3   Group                    346 non-null    object
 4   Causes of deaths         346 non-null    object
 5   Rationale for inclusion  346 non-null    object
 6   Coluna2                  346 non-null    object
 7   CID_PAI                  346 non-null    object
dtypes: object(8)
memory usage: 21.8+ KB


##### CIDs doenças evitáveis - MS/SVS

In [29]:
# Exibição de informações da tabela de CID evitáveis de acordo com o MS/SVS
cid_evit_svs = pd.read_excel('downloads_outros/SVS_cid_evitavel_clean.xlsx')
cid_evit_svs.head()

,CID10_EVIT,CID10_EVIT_CLASS,cid10_evit_nome,cid10_evit_atributo,cid10_evit_cod_pai
0,Causas Evitáveis,Reduzíveis pelas ações de imunoprevenção,Tuberculose do sistema nervoso,(A17),A17
1,Causas Evitáveis,Reduzíveis pelas ações de imunoprevenção,Tuberculose miliar,(A19),A19
2,Causas Evitáveis,Reduzíveis pelas ações de imunoprevenção,Tétano obstétrico,(A34),A34
3,Causas Evitáveis,Reduzíveis pelas ações de imunoprevenção,Outros tipos de tétano,(A35),A35
4,Causas Evitáveis,Reduzíveis pelas ações de imunoprevenção,Difteria,(A36),A36


##### População - Estimada SIDRA/IBGE

In [3]:
# Definir variáveis para carga de dados do IBGE
    # tabela de estimativa da população = 6579
    # nível territorial = Brasil(1), Região(2), Estados(3), Municípios(6)
    # geocode = 88, traz códigos e nomes de Estados e Municípios
    # período = 2018 a 2021
    # Carga de dados pelo SIDRA do IBGE

# Importa os dados do SIDRA
popest_ibge = sidrapy.get_table(table_code= "6579",
                            territorial_level = "6",
                            ibge_territorial_code = "all",
                            period = "2018-2024")


In [4]:
popest_ibge.head(10)

,NC,NN,MC,MN,V,D1C,D1N,D2C,D2N,D3C,D3N
0,Nível Territorial (Código),Nível Territorial,Unidade de Medida (Código),Unidade de Medida,Valor,Município (Código),Município,Ano (Código),Ano,Variável (Código),Variável
1,6,Município,45,Pessoas,23167,1100015,Alta Floresta D'Oeste - RO,2018,2018,9324,População residente estimada
2,6,Município,45,Pessoas,22945,1100015,Alta Floresta D'Oeste - RO,2019,2019,9324,População residente estimada
3,6,Município,45,Pessoas,22728,1100015,Alta Floresta D'Oeste - RO,2020,2020,9324,População residente estimada
4,6,Município,45,Pessoas,22516,1100015,Alta Floresta D'Oeste - RO,2021,2021,9324,População residente estimada
5,6,Município,45,Pessoas,22853,1100015,Alta Floresta D'Oeste - RO,2024,2024,9324,População residente estimada
6,6,Município,45,Pessoas,106168,1100023,Ariquemes - RO,2018,2018,9324,População residente estimada
7,6,Município,45,Pessoas,107863,1100023,Ariquemes - RO,2019,2019,9324,População residente estimada
8,6,Município,45,Pessoas,109523,1100023,Ariquemes - RO,2020,2020,9324,População residente estimada
9,6,Município,45,Pessoas,111148,1100023,Ariquemes - RO,2021,2021,9324,População residente estimada


In [47]:
# Salvar arquivo de população estimada do IBGE
ibge_sidra_pop_est = popest_ibge.to_csv('downloads_outros/ibge_sidra_pop_est.csv')

##### População - Projeção SIDRA/IBGE

In [43]:
# Definir variáveis para carga de dados do IBGE
    # tabela de projeção da população = 7358
    # nível territorial = Brasil(1), Região(2), Estados(3), Municípios(6) - não há dados por município para projeção
    # período = 2018 a 2021
    # classificação = 2 para sexo; os demais trazem faixas etárias
    # Carga de dados pelo SIDRA do IBGE

# Importa os dados do SIDRA
pop_proj_ibge = sidrapy.get_table(table_code= "7358",
                            territorial_level = "3",
                            ibge_territorial_code = "all",
                            period = "2018-2024",
							classificacao='2[all]|287[100362,93070,93084,93085,93086,93087,93088,93089,93090,93091,93092,93093,93094,93095,93096,93097,93098,113623,49109,49110]|1933[49034,49035,49036,49037,49038]')


In [44]:
pop_proj_ibge.head(10)

,NC,NN,MC,MN,V,D1C,D1N,D2C,D2N,D3C,D3N,D4C,D4N,D5C,D5N,D6C,D6N
0,Nível Territorial (Código),Nível Territorial,Unidade de Medida (Código),Unidade de Medida,Valor,Unidade da Federação (Código),Unidade da Federação,Ano (Código),Ano,Variável (Código),Variável,Sexo (Código),Sexo,Idade (Código),Idade,Ano (Código),Ano
1,3,Unidade da Federação,45,Pessoas,..,11,Rondônia,2018,2018,606,População,6794,Total,100362,Total,0,
2,3,Unidade da Federação,45,Pessoas,..,12,Acre,2018,2018,606,População,6794,Total,100362,Total,0,
3,3,Unidade da Federação,45,Pessoas,..,13,Amazonas,2018,2018,606,População,6794,Total,100362,Total,0,
4,3,Unidade da Federação,45,Pessoas,..,14,Roraima,2018,2018,606,População,6794,Total,100362,Total,0,
5,3,Unidade da Federação,45,Pessoas,..,15,Pará,2018,2018,606,População,6794,Total,100362,Total,0,
6,3,Unidade da Federação,45,Pessoas,..,16,Amapá,2018,2018,606,População,6794,Total,100362,Total,0,
7,3,Unidade da Federação,45,Pessoas,..,17,Tocantins,2018,2018,606,População,6794,Total,100362,Total,0,
8,3,Unidade da Federação,45,Pessoas,..,21,Maranhão,2018,2018,606,População,6794,Total,100362,Total,0,
9,3,Unidade da Federação,45,Pessoas,..,22,Piauí,2018,2018,606,População,6794,Total,100362,Total,0,


In [46]:
# Salvar arquivo de população estimada do IBGE
ibge_sidra_pop_proj = pop_proj_ibge.to_csv('downloads_outros/ibge_sidra_pop_proj.csv')

### Tratamento de Dados

#### Tratamento de População Estimada

In [5]:
# Carga de arquivo de População Estimada do IBGE
ibge_sidra_pop_est = pd.read_csv('downloads_outros/ibge_sidra_pop_est.csv')

In [6]:
# Renomear as colunas
popest_ibge_clean = ibge_sidra_pop_est.rename(columns={"D1C": "MUNCOD", "D1N": "MUNNOME", "V": "POPULACAO", "D2N": "ANO"})

# Excluir as colunas que não serão utilizadas
popest_ibge_clean = popest_ibge_clean.drop(columns=['NC','NN','MC','MN',"D2C", "D3C", "D3N"], axis=1)

# Excluir a primeira linha
popest_ibge_clean = popest_ibge_clean.iloc[1:]

# Resetar o índice
popest_ibge_clean.reset_index(drop=True, inplace=True)

# Copiar os 2 primeiros caracteres da coluna MUNCOD para a nova coluna UFCOD
popest_ibge_clean['UFCOD'] = popest_ibge_clean['MUNCOD'].str[:2]

#trazer a coluna ufnome para o DataFrame popest_ibge_clean
# Dar carga em tabela auxiliar com nomes das UFs, para posterior merge com a tabela de população
uf = pd.read_csv('downloads_outros/ibge_ufnome.csv', sep=';')
uf.UFCOD = uf.UFCOD.astype(str)
popest_ibge_clean = pd.merge(popest_ibge_clean, uf, on='UFCOD')


##### Arquivo pronto


In [8]:
# Arquivo de População Estimada Tratado (pronto para merge)
popest_ibge_clean.head()

,Unnamed: 0,POPULACAO,MUNCOD,MUNNOME,ANO,UFCOD,UFNOM
0,1,23167,1100015,Alta Floresta D'Oeste - RO,2018,11,Rondônia
1,2,22945,1100015,Alta Floresta D'Oeste - RO,2019,11,Rondônia
2,3,22728,1100015,Alta Floresta D'Oeste - RO,2020,11,Rondônia
3,4,22516,1100015,Alta Floresta D'Oeste - RO,2021,11,Rondônia
4,5,22853,1100015,Alta Floresta D'Oeste - RO,2024,11,Rondônia


#### Tratamento de População Projeção

In [48]:
pop_proj_ibge.head()

,NC,NN,MC,MN,V,D1C,D1N,D2C,D2N,D3C,D3N,D4C,D4N,D5C,D5N,D6C,D6N
0,Nível Territorial (Código),Nível Territorial,Unidade de Medida (Código),Unidade de Medida,Valor,Unidade da Federação (Código),Unidade da Federação,Ano (Código),Ano,Variável (Código),Variável,Sexo (Código),Sexo,Idade (Código),Idade,Ano (Código),Ano
1,3,Unidade da Federação,45,Pessoas,..,11,Rondônia,2018,2018,606,População,6794,Total,100362,Total,0,
2,3,Unidade da Federação,45,Pessoas,..,12,Acre,2018,2018,606,População,6794,Total,100362,Total,0,
3,3,Unidade da Federação,45,Pessoas,..,13,Amazonas,2018,2018,606,População,6794,Total,100362,Total,0,
4,3,Unidade da Federação,45,Pessoas,..,14,Roraima,2018,2018,606,População,6794,Total,100362,Total,0,


In [2]:
# Renomear as colunas
popest_ibge_clean = popest_ibge.rename(columns={"D1C": "UFCOD", "D1N": "UFNOME", "V": "POPULACAO", "D2N": "ANO"})

# Excluir as colunas que não serão utilizadas
popest_ibge_clean = popest_ibge_clean.drop(columns=['NC','NN','MC','MN',"D2C", "D3C", "D3N"], axis=1)

# Excluir a primeira linha
popest_ibge_clean = popest_ibge_clean.iloc[1:]

# Resetar o índice
popest_ibge_clean.reset_index(drop=True, inplace=True)

# Copiar os 2 primeiros caracteres da coluna MUNCOD para a nova coluna UFCOD
popest_ibge_clean['UFCOD'] = popest_ibge_clean['MUNCOD'].str[:2]

#trazer a coluna ufnome para o DataFrame popest_ibge_clean
# Dar carga em tabela auxiliar com nomes das UFs, para posterior merge com a tabela de população
uf = pd.read_csv('downloads_outros/ibge_ufnome.csv', sep=';')
uf.UFCOD = uf.UFCOD.astype(str)
popest_ibge_clean = pd.merge(popest_ibge_clean, uf, on='UFCOD')

# Exibir as primeiras linhas do DataFrame atualizado
popest_ibge_clean.head()

NameError: name 'popest_ibge' is not defined

#### Tratamento do arquivo principal

O dicionário do SIM está em "./Estrutura_SIM_para_CD.pdf"

Trabalhamos o CID presente no campo CAUSABAS.

Em razão do escopo da População Estimada (que apresenta dados por Município, mas não de outras variáveis de desigualdade) a primeira versão será um recorte da base DOBR com os dados de CID ('CAUSABAS'), idade (a partir do tratamento de 'DTNASC'), e de Município (a partir do tratamento de 'CODMUNRES').



In [4]:
# Função para adicionar o campo de ano
def add_year_column(file_path):
    year = file_path[-12:-8]
    df = pd.read_parquet(file_path)
    df['ANO'] = year
    return df

# Lista de arquivos parquet
arquivos_parquet = ['downloads_sim_original/DOBR2021.parquet', 'downloads_sim_original/DOBR2022.parquet']

# Ler, adicionar o campo de ano e concatenar os arquivos parquet
dobr_df = pd.concat([add_year_column(arquivo) for arquivo in arquivos_parquet])

# Resetar o índice
dobr_df.reset_index(drop=True, inplace=True)

# Exibir as primeiras linhas do DataFrame atualizado
dobr_df.head()

,ORIGEM,TIPOBITO,DTOBITO,HORAOBITO,NATURAL,CODMUNNATU,DTNASC,IDADE,SEXO,RACACOR,...,TPRESGINFO,TPNIVELINV,NUDIASINF,DTCADINF,MORTEPARTO,DTCONCASO,FONTESINF,ALTCAUSA,CONTADOR,ANO
0,1,2,28032021,1310,813,130170,02021943,478,2,4,...,,,,,,,,,570001,2021
1,1,2,26032021,2100,,,01091961,459,1,1,...,,,,,,,,,570002,2021
2,1,2,04012021,0400,825,250750,01071946,474,2,1,...,,,,,,,,,570003,2021
3,1,2,31012021,1700,825,251150,24101956,464,2,4,...,,,,,,,,,570004,2021
4,1,2,20022021,,825,251500,21041943,477,1,,...,,,,,,,,,570005,2021


In [3]:
# # Lista de arquivos parquet
# arquivos_parquet = [
#     # 'downloads_sim_original/DOBR2019.parquet',
#     # 'downloads_sim_original/DOBR2020.parquet',
#     'downloads_sim_original/DOBR2021.parquet',
#     'downloads_sim_original/DOBR2022.parquet'
# ]

# # Ler e concatenar os arquivos parquet
# dobr_parquet = pd.concat([pd.read_parquet(arquivo) for arquivo in arquivos_parquet])

# # Transformar em DataFrame
# dobr_df = pd.DataFrame(dobr_parquet)


##### Reduzir o arquivo principal apenas para campos referenciados para População Estimada

In [11]:
# Filtro de campos a serem utilizados - versão 1
dobr_cid_mun_idade = dobr_df[['CAUSABAS','DTNASC', 'DTOBITO','CODMUNRES']]
dobr_cid_mun_idade.head()


,CAUSABAS,DTNASC,DTOBITO,CODMUNRES
0,B342,02021943,28032021,110020
1,B342,01091961,26032021,110020
2,I64,01071946,04012021,250750
3,C785,24101956,31012021,250750
4,C189,21041943,20022021,250750


##### Tratamento de datas

In [12]:
# Função para transformar datas e excluir registros inválidos
def transform_date_columns(df, date_columns):
    invalid_rows = []
    
    for date_column in date_columns:
        for index, row in df.iterrows():
            try:
                df.at[index, f'{date_column}_trat'] = pd.to_datetime(row[date_column], format='%d%m%Y').strftime('%d/%m/%Y')
            except ValueError:
                invalid_rows.append(index)
    
    # Excluir as linhas identificadas em invalid_rows
    df = df.drop(invalid_rows)
    
    # Apresentar a conta das linhas com erros
    print(f"Linhas com erro: {len(invalid_rows)}")
    
    return df

# Transformar os campos DTNASC e DTOBITO
dobr_cid_mun_idade = transform_date_columns(dobr_cid_mun_idade, ['DTNASC', 'DTOBITO'])

# Verificar a nova base
print(dobr_cid_mun_idade.info())
dobr_cid_mun_idade.head()

/tmp/ipykernel_64801/3653329788.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.at[index, f'{date_column}_trat'] = pd.to_datetime(row[date_column], format='%d%m%Y').strftime('%d/%m/%Y')
/tmp/ipykernel_64801/3653329788.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.at[index, f'{date_column}_trat'] = pd.to_datetime(row[date_column], format='%d%m%Y').strftime('%d/%m/%Y')


Linhas com erro: 6891
<class 'pandas.core.frame.DataFrame'>
Index: 3370024 entries, 0 to 3376914
Data columns (total 6 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   CAUSABAS      object
 1   DTNASC        object
 2   DTOBITO       object
 3   CODMUNRES     object
 4   DTNASC_trat   object
 5   DTOBITO_trat  object
dtypes: object(6)
memory usage: 180.0+ MB
None


,CAUSABAS,DTNASC,DTOBITO,CODMUNRES,DTNASC_trat,DTOBITO_trat
0,B342,02021943,28032021,110020,02/02/1943,28/03/2021
1,B342,01091961,26032021,110020,01/09/1961,26/03/2021
2,I64,01071946,04012021,250750,01/07/1946,04/01/2021
3,C785,24101956,31012021,250750,24/10/1956,31/01/2021
4,C189,21041943,20022021,250750,21/04/1943,20/02/2021


In [17]:
dobr_cid_mun_idade = dobr_cid_mun_idade.to_csv('files_clean/dobr_cid_mun_idade.csv')

In [15]:
# pegar 20 linhas aleatórias
df_teste = dobr_cid_mun_idade.sample(20)
df_teste

,CAUSABAS,DTNASC,DTOBITO,CODMUNRES,DTNASC_trat,DTOBITO_trat
2009139,I255,12091965,13092022,355030,12/09/1965,13/09/2022
299471,I694,28111949,10012021,130010,28/11/1949,10/01/2021
3011684,C169,10021958,07112022,353070,10/02/1958,07/11/2022
2494978,E875,01101952,23032022,412810,01/10/1952,23/03/2022
617412,B342,07051972,03122021,500770,07/05/1972,03/12/2021
205173,K830,26081940,09032021,351870,26/08/1940,09/03/2021
648925,K579,08061940,31102021,261070,08/06/1940,31/10/2021
1847832,I212,17041951,07042022,313190,17/04/1951,07/04/2022
1130387,C187,02031969,02032021,261160,02/03/1969,02/03/2021
3212705,K701,27081973,03052022,410690,27/08/1973,03/05/2022


##### Criação de faixas de idade

In [10]:
dobr_cid_mun_idade = pd.read_csv('files_clean/dobr_cid_mun_idade.csv')

In [11]:
# Calcular a diferença entre DTNASC_trat e DTOBITO_trat como OBITO_IDADE
dobr_cid_mun_idade['OBITO_IDADE'] = (pd.to_datetime(dobr_cid_mun_idade['DTOBITO_trat'], format='%d/%m/%Y') - pd.to_datetime(dobr_cid_mun_idade['DTNASC_trat'], format='%d/%m/%Y')).dt.days // 365

# Gerar o campo IDADE_FAIXA com agrupamentos de 5 em 5 anos
bins = list(range(0, 85, 5)) + [float('inf')]
labels = [f'{i}-{i+4}' for i in range(0, 80, 5)] + ['80+']
dobr_cid_mun_idade['IDADE_FAIXA'] = pd.cut(dobr_cid_mun_idade['OBITO_IDADE'], bins=bins, labels=labels, right=False)

# Verificar as primeiras linhas do DataFrame atualizado
dobr_cid_mun_idade.head()

,Unnamed: 0,CAUSABAS,DTNASC,DTOBITO,CODMUNRES,DTNASC_trat,DTOBITO_trat,OBITO_IDADE,IDADE_FAIXA
0,0,B342,2021943,28032021,110020,02/02/1943,28/03/2021,78,75-79
1,1,B342,1091961,26032021,110020,01/09/1961,26/03/2021,59,55-59
2,2,I64,1071946,4012021,250750,01/07/1946,04/01/2021,74,70-74
3,3,C785,24101956,31012021,250750,24/10/1956,31/01/2021,64,60-64
4,4,C189,21041943,20022021,250750,21/04/1943,20/02/2021,77,75-79


In [12]:
# GroupBy para Faixas de Idade
dobr_cid_mun_idade.groupby('IDADE_FAIXA').size()


/tmp/ipykernel_1787/1158828235.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dobr_cid_mun_idade.groupby('IDADE_FAIXA').size()


IDADE_FAIXA
0-4        75065
5-9         6012
10-14       7824
15-19      29893
20-24      51956
25-29      56346
30-34      64123
35-39      84185
40-44     109532
45-49     134695
50-54     173572
55-59     233329
60-64     284233
65-69     333448
70-74     360556
75-79     362015
80+      1003240
dtype: int64

In [25]:
# Excluir as faixas de idade '75-79' e '80+'
dobr_clean = dobr_cid_mun_idade[~dobr_cid_mun_idade['IDADE_FAIXA'].isin(['75-79', '80+'])]
dobr_clean.reset_index(drop=True, inplace=True)


In [26]:
# Criar o campo "ANO" com os últimos 4 caracteres de "DTOBITO_trat"
dobr_clean['ANO'] = dobr_clean['DTOBITO_trat'].str[-4:]

# Listar e filtrar colunas a serem utilizadas
dobr_clean = dobr_clean[['CAUSABAS', 'CODMUNRES', 'IDADE_FAIXA', 'ANO']]

# Converter todos os campos para string
dobr_clean = dobr_clean.astype(str)


/tmp/ipykernel_1787/1215537056.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dobr_clean['ANO'] = dobr_clean['DTOBITO_trat'].str[-4:]


##### Arquivo pronto


In [27]:
# Verificar as primeiras linhas do DataFrame atualizado
dobr_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2004769 entries, 0 to 2004768
Data columns (total 4 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   CAUSABAS     object
 1   CODMUNRES    object
 2   IDADE_FAIXA  object
 3   ANO          object
dtypes: object(4)
memory usage: 61.2+ MB


Resultados:

A partir do critério de qualidade dos dados de DTNASC e DTOBITO, foram excluídos:
* 6.891 registros que não apresentavam o campo compatível com o formato data
* 362.015 registro de pessoas com idade de óbito entre 75-79 anos
* 1.003.240 registros de pessoas com idade de óbito 80 ou mais


O Dataframe principal tem no total 2.004.769  registros.

### Merge de dados de CID e População

In [29]:
print(dobr_clean['CAUSABAS'].info())
print(cid_evit_ocde['CAUSABAS'].info())

<class 'pandas.core.series.Series'>
RangeIndex: 2004769 entries, 0 to 2004768
Series name: CAUSABAS
Non-Null Count    Dtype 
--------------    ----- 
2004769 non-null  object
dtypes: object(1)
memory usage: 15.3+ MB


In [34]:
df_a = dobr_clean
column_a = 'CAUSABAS'
df_b = cid_evit_ocde
column_b = 'CID_AVOID'
column_return = 'CID_PAI'



# Função para verificar a correspondência e retornar a classificação
def verificar_correspondencia(row):
    for index, row_b in df_b.iterrows():
        if row[column_a].startswith(row_b[column_b]):
            return row_b[column_return]
    return 'causa não evitável'

# Aplicando a função a cada linha do DataFrame A
df_a['classificacao'] = df_a.apply(verificar_correspondencia, axis=1)

print(df_a)

KeyboardInterrupt: 

In [2]:
# Exibição de informações da tabela de CID evitáveis de acordo com o MS/SVS
cid_evit_svs = pd.read_excel('downloads_outros/SVS_cid_evitavel_clean.xlsx')
cid_evit_svs.head()

,CID10_EVIT,CID10_EVIT_CLASS,cid10_evit_nome,cid10_evit_atributo,cid10_evit_cod_pai
0,Causas Evitáveis,Reduzíveis pelas ações de imunoprevenção,Tuberculose do sistema nervoso,(A17),A17
1,Causas Evitáveis,Reduzíveis pelas ações de imunoprevenção,Tuberculose miliar,(A19),A19
2,Causas Evitáveis,Reduzíveis pelas ações de imunoprevenção,Tétano obstétrico,(A34),A34
3,Causas Evitáveis,Reduzíveis pelas ações de imunoprevenção,Outros tipos de tétano,(A35),A35
4,Causas Evitáveis,Reduzíveis pelas ações de imunoprevenção,Difteria,(A36),A36


In [3]:
# Exibição de informações da tabela de CID evitáveis de acordo com a OCDE
cid_evit_ocde = pd.read_excel('downloads_outros/OCDE_avoidable_mortality.xlsx')
cid_evit_ocde.head()

,Avoidable mortality,Preventable mortality,Treatable mortality,Group,Causes of deaths,Rationale for inclusion,Coluna2,ICD.IO
0,Avoidable mortality,Preventable mortality,NaN,Infectious diseases,Intestinal diseases,Most of these infections can be prevented thro...,(A00-A09),A0
1,Avoidable mortality,Preventable mortality,NaN,Infectious diseases,"Diphtheria, Tetanus, Poliomyelitis",Most of these infections can be prevented thro...,"(A35, A36, A80)",A35
2,Avoidable mortality,Preventable mortality,NaN,Infectious diseases,"Diphtheria, Tetanus, Poliomyelitis",Most of these infections can be prevented thro...,"(A35, A36, A80)",A36
3,Avoidable mortality,Preventable mortality,NaN,Infectious diseases,"Diphtheria, Tetanus, Poliomyelitis",Most of these infections can be prevented thro...,"(A35, A36, A80)",A80
4,Avoidable mortality,Preventable mortality,NaN,Infectious diseases,Whooping cough,Most of these infections can be prevented thro...,(A37),A37


In [17]:
popest_ibge_clean.head()


,Unnamed: 0,POPULACAO,MUNCOD,MUNNOME,ANO,UFCOD,UFNOM
0,1,23167,1100015,Alta Floresta D'Oeste - RO,2018,11,Rondônia
1,2,22945,1100015,Alta Floresta D'Oeste - RO,2019,11,Rondônia
2,3,22728,1100015,Alta Floresta D'Oeste - RO,2020,11,Rondônia
3,4,22516,1100015,Alta Floresta D'Oeste - RO,2021,11,Rondônia
4,5,22853,1100015,Alta Floresta D'Oeste - RO,2024,11,Rondônia


In [16]:
dobr_clean.head()

,CAUSABAS,CODMUNRES,IDADE_FAIXA,ANO
0,B342,110020,55-59,2021
1,I64,250750,70-74,2021
2,C785,250750,60-64,2021
3,I64,250320,70-74,2021
4,J189,250320,70-74,2021


In [93]:
# Realizar o merge dos dataframes
merged_df = pd.merge(dobr_clean, popest_ibge_clean, left_on=['CODMUNRES', 'ANO'], right_on=['MUNCOD', 'ANO'], how='left')

# Exibir as primeiras linhas do dataframe resultante
print(merged_df.head())

  CAUSABAS CODMUNRES IDADE_FAIXA   ANO  Unnamed: 0 POPULACAO MUNCOD MUNNOME  \
0     B342    110020       55-59  2021         NaN       NaN    NaN     NaN   
1     I64     250750       70-74  2021         NaN       NaN    NaN     NaN   
2     C785    250750       60-64  2021         NaN       NaN    NaN     NaN   
3     I64     250320       70-74  2021         NaN       NaN    NaN     NaN   
4     J189    250320       70-74  2021         NaN       NaN    NaN     NaN   

  UFCOD UFNOM  
0   NaN   NaN  
1   NaN   NaN  
2   NaN   NaN  
3   NaN   NaN  
4   NaN   NaN  


In [ ]:
# Create a new column in dobr_clean to indicate if CAUSABAS starts with any value in cid_evit_ocde.ICD.IO
dobr_clean['is_avoidable'] = dobr_clean['CAUSABAS'].apply(lambda x: any(x.startswith(icd) for icd in cid_evit_ocde['ICD.IO']))

# Display the updated dataframe
dobr_clean.head()

In [ ]:
#pesquisar se dobr_clean.causabas possui os primeiros caracteres de cid_evit_ocde.icd.10
#pesquisar se dobr_clean.causabas possui os primeiros caracteres de cid_evit_ocde.icd.10



In [20]:
#Fazer um merge entre dobr_clean e cid_evit_ocde, pesquisando se a coluna CAUSABAS começa com o valor da coluna ICD.IO
# Realizar o merge dos dataframes
merged_df = pd.merge(dobr_clean, cid_evit_ocde, left_on=['CAUSABAS'], right_on=['ICD.IO'], how='left')

In [21]:
merged_df.head()

,CAUSABAS,CODMUNRES,IDADE_FAIXA,ANO,Avoidable mortality,Preventable mortality,Treatable mortality,Group,Causes of deaths,Rationale for inclusion,Coluna2,ICD.IO
0,B342,110020,55-59,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,I64,250750,70-74,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C785,250750,60-64,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,I64,250320,70-74,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,J189,250320,70-74,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Realizar o merge dos dataframes
merged_df = pd.merge(dobr_clean, cid_evit_ocde, left_on=['CAUSABAS'], right_on=['MUNCOD', 'ANO'], how='left')

# Exibir as primeiras linhas do dataframe resultante
print(merged_df.head())

In [ ]:
# prompt: juntar dados de df e cid10_df
file_left1 = dobr_clean[['CAUSABAS', 'CODMUNRES', 'IDADE_FAIXA','ANO']]
file_right1 = popest_ibge_clean [['UFCOD', 'UFNOM', 'MUNCOD', 'MUNNOME', 'ANO', 'POPULACAO' ]] 
file_right2 = cid_evit_ocde [['Avoidable mortality', 'Preventable mortality', 'Treatable mortality', 
                              'Group', 'Causes of deaths', 'ICD.IO']] 
file_right3 = cid_evit_svs [['CID10_EVIT', 'CID10_EVIT_CLASS', 'cid10_evit_nome','cid10_evit_cod_pai']]

# Merge the dataframes
file_left2 = pd.merge(file_left1, file_right1, left_on='CAUSABAS', right_on='cid10_cod', how='left') # Trazer nome do CID
merged1 = pd.merge(file_left2, file_right2, left_on='CODMUNOCOR', right_on='MUNCOD', how='left')
merged = pd.merge(merged1, file_right2, left_on='CODMUNRES', right_on='MUNCOD', how='left')

# Renomear colunas
merged.rename(columns={'MUNNOME_x':'MUNNOME_OCOR','MUNNOME_y':'MUNNOME_RES'},inplace=True)

# Excluir colunas
merged_clean = merged.drop(columns=['MUNCOD_x','MUNCOD_y','cid10_cod'])
# merged1 = pd.merge(file_left2, file_right2, left_on='CODMUNOCOR', right_on='MUNCOD', how='left').rename(columns={'MUNNOME':'MUNNOME_OCOR'},inplace=True) # Trazer nome de município de ocorrência
# merged = pd.merge(merged1, file_right2, left_on='CODMUNRES', right_on='MUNCOD', how='left').rename(columns={'MUNNOME':'MUNNOME_RES'},inplace=True) # Trazer nome de município de residência
# Display the merged dataframe
# merged_df = pd.DataFrame(merged)
print(merged_clean.info())
merged_clean.head()

In [46]:
# Exibir as colunas dos dataframes do arquivo principal e dos demais arquivos de população estimada e CID evitáveis
cid_evit_svs.head()

,CID10_EVIT,CID10_EVIT_CLASS,cid10_evit_nome,cid10_evit_atributo,cid10_evit_cod_pai
0,Causas Evitáveis,Reduzíveis pelas ações de imunoprevenção,Tuberculose do sistema nervoso,(A17),A17
1,Causas Evitáveis,Reduzíveis pelas ações de imunoprevenção,Tuberculose miliar,(A19),A19
2,Causas Evitáveis,Reduzíveis pelas ações de imunoprevenção,Tétano obstétrico,(A34),A34
3,Causas Evitáveis,Reduzíveis pelas ações de imunoprevenção,Outros tipos de tétano,(A35),A35
4,Causas Evitáveis,Reduzíveis pelas ações de imunoprevenção,Difteria,(A36),A36


### Seleção e tratamento de variáveis da base primária

#### Carregar arquivos concatenados como dataframe

# Etapa 3 - Aplicação de Modelo do HEAT ao Bloco

# Etapa 4 - Reunião de alinhamento - ajustes

# Etapa 5 - Preparação para publicação de resultados